## Objective
The purpose of this product is to provide an automated service that recommends leads to a user given their current customer list (Portfolio).

## Contextualization
Some companies would like to know who are the other companies in a given market (population) that are most likely to become their next customers. That is, your solution must find in the market who are the most adherent leads given the characteristics of the customers present in the user's portfolio.

In addition, your solution must be user agnostic. Any user with a list of customers who want to explore this market can extract value from the service.

Para o desafio, deverão ser consideradas as seguintes bases:

Mercado: Base com informações sobre as empresas do Mercado a ser considerado. Portfolio 1: Ids dos clientes da empresa 1 Portfolio 2: Ids dos clientes da empresa 2 Portfolio 3: Ids dos clientes da empresa 3

Obs: todas as empresas(ids) dos portfolios estão contidos no Mercado(base de população).

In [112]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="darkgrid",font_scale=0.6)
plt.rcParams["figure.figsize"] = (15,8)

In [321]:
def get_info(df):
    '''
    Get basic info from data set
    '''
    print('Shape :: ', df.shape)
    return pd.DataFrame({'types': df.dtypes,
                            'nan': df.isna().sum(), 
                            'nan%': ((df.isna().sum()/len(df))*100).round(1), 
                            'unique':df.nunique()
                            })

def dropna_cols(df,info,max_na): 
    '''
    Function to drop columns above threshold max_na
    '''
    return df.drop(info[info['nan%']  > max_na].index, axis=1).copy()

def categorical_types(df):
    '''
    Our model requires to know where are the positions of each categorical variable
    Here we extract this info!

    returns
    : all_features - complete set of columns
    : categ_features -  boolean and object features 
    : num_features - numeric features
    : categ_bool - the whole index of columns stating True for categorical variables
    '''
    
    all_features = df.columns.to_list()
      
    categ_features = df.select_dtypes(['bool','object']).columns.to_list()
    num_features = df.select_dtypes(['int64','float']).columns.to_list()

    categ_loc = [df.columns.get_loc(x) for x in categ_features]
    categ_func = lambda x: True if x in categ_features else False
    categ_bool = [categ_func(x) for x in all_features]
    categ_idx = dict(zip(categ_features, categ_bool))

    print(f'{df.shape} \n\nNum:{num_features} \n\nCateg: {categ_features}')
    
    return (all_features,categ_features,num_features,categ_bool)

def heatmap_plot(df, cols): 
    crosstable = contigency_table(df, cols)
    sns.heatmap(
    crosstable.apply(lambda x : x / x.sum(), axis=1),
    cmap='BuPu',
    square=True,
    cbar_kws = {'fraction' : 0.02}
    )
    return plt.show() 

def group_unique(df,cols):
    aux = df[cols].drop_duplicates()\
                  .groupby([cols[0],cols[1]])[cols[2]]\
                  .apply(list)\
                  .reset_index()
    return aux.apply(lambda x: (',\n'.join([str(s) for s in x[cols[2]]])), axis = 1)

In [357]:
def contigency_table(df, cols,na=False,ax=0,normalize=False):
    '''
    cols :: this is a list of lists 
            it must have length of 2
            col[0] contains the values to group by in the index
            col[1] contains the values to group by in the columns
    na:: Do not include columns whose entries are all NaN
    ax:: Axis to consider the bar styling % 
        if 0 ->(rows) 
        if 1 -> (columns) 
    normalize:: Normalize by dividing all values by the sum of values
                If passed ‘all’ or True, will normalize over all values.
                If passed ‘index’ will normalize over each row.
                If passed ‘columns’ will normalize over each column.
                If margins is True, will also normalize margin values.  

    '''

    if len(cols)<=2:
        if len(cols) == 1:
            crosstable = pd.crosstab(
                index = df[cols[0][0]], 
                columns="Total",
                dropna=na,
                normalize = normalize)
        elif len(cols) == 2: 
            if len( cols[0]) == 1 and len( cols[1]) == 1:
                crosstable = pd.crosstab(
                    index = df[cols[0][0]], 
                    columns = df[cols[1][0]],  
                    rownames= cols[0],
                    colnames= cols[1],
                    dropna=na,
                    margins=True, 
                    margins_name="Total",
                    normalize = normalize)
            elif len( cols[0]) == 1 and len( cols[1]) == 2:
                crosstable = pd.crosstab(
                    index = df[cols[0][0]], 
                    columns =[df[cols[1][0]], df[cols[1][1]]], 
                    rownames= cols[0],
                    colnames= cols[1],
                    dropna=na,
                    margins=True, 
                    margins_name="Total",
                    normalize = normalize)
            elif len( cols[0]) == 2 and len( cols[1]) == 1:
                crosstable = pd.crosstab(
                    index =[df[cols[0][0]], df[cols[0][1]]], 
                    columns = df[cols[1][0]], 
                    rownames= cols[0],
                    colnames= cols[1],
                    dropna=na,
                    margins=True, 
                    margins_name="Total",
                    normalize = normalize)    
            elif len( cols[0]) == 2 and len( cols[1]) == 2:
                crosstable = pd.crosstab(
                    index =[df[cols[0][0]],df[cols[0][1]]],
                    columns =[ df[cols[1][0]], df[cols[1][1]]], 
                    rownames=cols[0],
                    colnames= cols[1],
                    dropna=na,
                    margins=True, 
                    margins_name="Total",
                    normalize = normalize)   
        return crosstable.style.bar(color='#3d66af', axis = ax)#.background_gradient( axis=None)

    else:
        print('The parameter cols must be a list of lists, it has length of 2!\
               \ncols[0] = [feat1, feat2] || cols[1] = [feat3, feat4] ')
        return None

In [356]:
def values_contigency_table(df, cols, values = None, aggfunc = None, na=False,ax=0, normalize = False):

    '''
    cols :: this is a list of lists - it must have length of 2!
            col[0] contains the names of the columns to group by in the index
            col[1] contains the names of the columns to group by in the columns
    values :: array of values to aggregate according to the factors.
    aggfunc = 'mean'|'sum'
    na:: Do not include columns whose entries are all NaN
    ax:: Axis to consider the bar styling % 
        if 0 ->(rows) 
        if 1 -> (columns) 
    normalize:: Normalize by dividing all values by the sum of values
                If passed ‘all’ or True, will normalize over all values.
                If passed ‘index’ will normalize over each row.
                If passed ‘columns’ will normalize over each column.
                If margins is True, will also normalize margin values.  

    '''

    if len(cols)<=2:
        if len(cols) == 1:
            crosstable = pd.crosstab(
                index = df[cols[0][0]], 
                columns=aggfunc+'_'+values,
                values = df[values], 
                aggfunc = aggfunc,
                dropna=na,
                normalize = normalize).fillna('-')
        
        elif len(cols) == 2: 
            if len( cols[0]) == 1 and len( cols[1]) == 1:
                crosstable = pd.crosstab(
                    index = df[cols[0][0]], 
                    columns = df[cols[1][0]],
                    values = df[values], 
                    aggfunc = aggfunc,  
                    rownames= cols[0],
                    colnames= cols[1],
                    dropna=na,
                    margins=True, 
                    margins_name="Total",
                    normalize = normalize).fillna('-')
            elif len( cols[0]) == 1 and len( cols[1]) == 2:
                crosstable = pd.crosstab(
                    index = df[cols[0][0]], 
                    columns =[df[cols[1][0]], df[cols[1][1]]], 
                    values = df[values], 
                    aggfunc = aggfunc, 
                    rownames= cols[0],
                    colnames= cols[1],
                    dropna=na,
                    margins=True, 
                    margins_name="Total",
                    normalize = normalize).fillna('-')
            elif len( cols[0]) == 2 and len( cols[1]) == 1:
                crosstable = pd.crosstab(
                    index =[df[cols[0][0]], df[cols[0][1]]], 
                    columns = df[cols[1][0]], 
                    values = df[values], 
                    aggfunc = aggfunc,                 
                    rownames= cols[0],
                    colnames= cols[1],
                    dropna=na,
                    margins=True, 
                    margins_name="Total",
                    normalize = normalize).fillna('-')    
            elif len( cols[0]) == 2 and len( cols[1]) == 2:
                crosstable = pd.crosstab(
                    index =[df[cols[0][0]],df[cols[0][1]]],
                    columns =[ df[cols[1][0]], df[cols[1][1]]], 
                    values = df[values], 
                    aggfunc = aggfunc,                 
                    rownames=cols[0],
                    colnames= cols[1],
                    dropna=na,
                    margins=True, 
                    margins_name="Total",
                    normalize = normalize).fillna('-')
        return crosstable.style.background_gradient( axis=None)
    else:
        print('The parameter cols must be a list of lists, it has length of 2!\
               \ncols[0] = [feat1, feat2] || cols[1] = [feat3, feat4]. \
               \nThe parameter values is a string that indicates the name of the column.')
        return None
    

## Load Data

In [358]:
df_raw = pd.read_csv('estaticos_market.csv', index_col =0)

In [138]:
df_raw.head(3)

,id,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,fl_me,...,media_meses_servicos,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais
0,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...,True,SOCIEDADE EMPRESARIA LIMITADA,RN,ENTIDADES EMPRESARIAIS,INDUSTRIA DA CONSTRUCAO,CONSTRUÇÃO CIVIL,14.457534,10 a 15,False,...,43.738462,93.266667,19.166667,26.0,26.0,27.0,0.0,-3.703704,0.0,0
1,6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c4...,True,EMPRESARIO INDIVIDUAL,PI,OUTROS,SERVICOS DE ALOJAMENTO/ALIMENTACAO,SERVIÇO,1.463014,1 a 5,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02...,True,EMPRESARIO INDIVIDUAL,AM,OUTROS,"TRANSPORTE, ARMAZENAGEM E CORREIO",SERVIÇO,7.093151,5 a 10,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [139]:
nan_total = 100*(df_raw.isna().sum().sum()/(df_raw.shape[0]*df_raw.shape[1])).round(4)
print('Shape',df_raw.shape,' NaN%', nan_total)

Shape (462298, 181)  NaN% 66.34


In [359]:
info_DB = get_info(df_raw)

Shape ::  (462298, 181)


We set a threshold of accepted NaN values in order to be easier to work with the Data, and also reduce the number of imputation that can distorce the data.

In [360]:
max_percentual_na = 40
df = dropna_cols(df_raw,info_DB,max_percentual_na).copy()

## Avaliando localização

In [351]:
contigency_table(df,[['sg_uf']])

col_0,Total
sg_uf,
AC,20538
AM,82114
MA,127654
PI,76686
RN,98837
RO,56469


## Avaliando natureza jurídica

Uma vez que decidimos trabalhar com uma métrica que distingue  as variáves categórias ['fl_me','fl_sa', 'fl_epp', 'fl_mei', 'fl_ltda']  (elas podem trazer mais peso a mesma feature). Veja que  ['fl_optante_simei'] e  ['fl_optante_simples'] se tornam redundantes.


In [143]:
to_drop_fl = [ 'fl_optante_simei','fl_optante_simples']

df[['natureza_juridica_macro', 'de_natureza_juridica',
'fl_me', 'fl_sa', 'fl_epp',  'fl_mei', 'fl_ltda']].tail(5)

,natureza_juridica_macro,de_natureza_juridica,fl_me,fl_sa,fl_epp,fl_mei,fl_ltda
462293,OUTROS,EMPRESARIO INDIVIDUAL,False,False,False,True,False
462294,OUTROS,EMPRESARIO INDIVIDUAL,False,False,False,False,False
462295,ENTIDADES SEM FINS LUCRATIVOS,ASSOCIACAO PRIVADA,False,False,False,False,False
462296,ENTIDADES EMPRESARIAIS,SOCIEDADE EMPRESARIA LIMITADA,False,False,False,False,False
462297,OUTROS,EMPRESARIO INDIVIDUAL,False,False,False,True,False


In [183]:
contigency_table(df, cols = [['de_natureza_juridica'],['natureza_juridica_macro']])

natureza_juridica_macro,ADMINISTRACAO PUBLICA,CARGO POLITICO,ENTIDADES EMPRESARIAIS,ENTIDADES SEM FINS LUCRATIVOS,INSTITUICOES EXTRATERRITORIAIS,OUTROS,PESSOAS FISICAS,Total
de_natureza_juridica,,,,,,,,
ASSOCIACAO PRIVADA,0,0,0,29694,0,0,0,29694
AUTARQUIA ESTADUAL OU DO DISTRITO FEDERAL,40,0,0,0,0,0,0,40
AUTARQUIA FEDERAL,231,0,0,0,0,0,0,231
AUTARQUIA MUNICIPAL,123,0,0,0,0,0,0,123
CANDIDATO A CARGO POLITICO ELETIVO,0,1617,0,0,0,0,0,1617
CLUBE FUNDO DE INVESTIMENTO,0,0,0,0,0,8,0,8
COMUNIDADE INDIGENA,0,0,0,9,0,0,0,9
CONDOMINIO EDILICIO,0,0,0,2084,0,0,0,2084
CONSORCIO DE EMPREGADORES,0,0,1,0,0,0,0,1


## Avaliando setores

Ótimo! Agora vamos analisar quem são as empresas de acordo com o setor e informações requeridas pela receita (CNAE).

[ 'setor','de_ramo', 'nm_divisao', 'nm_segmento']

In [145]:
df[[ 'setor','de_ramo', 'nm_divisao', 'nm_segmento']].head(5)

,setor,de_ramo,nm_divisao,nm_segmento
0,CONSTRUÇÃO CIVIL,INDUSTRIA DA CONSTRUCAO,CONSTRUCAO DE EDIFICIOS,CONSTRUCAO
1,SERVIÇO,SERVICOS DE ALOJAMENTO/ALIMENTACAO,ALIMENTACAO,ALOJAMENTO E ALIMENTACAO
2,SERVIÇO,"TRANSPORTE, ARMAZENAGEM E CORREIO",TRANSPORTE TERRESTRE,TRANSPORTE ARMAZENAGEM E CORREIO
3,SERVIÇO,SERVICOS DIVERSOS,REPARACAO E MANUTENCAO DE EQUIPAMENTOS DE INFO...,OUTRAS ATIVIDADES DE SERVICOS
4,SERVIÇO,"SERVICOS PROFISSIONAIS, TECNICOS E CIENTIFICOS",SERVICOS DE ARQUITETURA E ENGENHARIA TESTES E ...,ATIVIDADES PROFISSIONAIS CIENTIFICAS E TECNICAS


In [89]:
for i in [ 'setor','de_ramo', 'nm_divisao', 'nm_segmento']:
    print(i, df[i].nunique())
    print(df[i].value_counts(dropna=False))
    print()

setor 5
COMERCIO            211224
SERVIÇO             189644
INDUSTRIA            30893
CONSTRUÇÃO CIVIL     25363
AGROPECUARIA          3247
NaN                   1927
Name: setor, dtype: int64

de_ramo 33
COMERCIO VAREJISTA                                   172404
SERVICOS DIVERSOS                                     60318
SERVICOS DE ALOJAMENTO/ALIMENTACAO                    30180
INDUSTRIA DA CONSTRUCAO                               25363
COMERCIO E REPARACAO DE VEICULOS                      22356
SERVICOS ADMINISTRATIVOS                              21326
BENS DE CONSUMO                                       21227
SERVICOS PROFISSIONAIS, TECNICOS E CIENTIFICOS        17371
COMERCIO POR ATACADO                                  16464
TRANSPORTE, ARMAZENAGEM E CORREIO                     15485
SERVICOS DE EDUCACAO                                  11311
SERVICOS DE SAUDE                                      9788
CULTURA, ESPORTE E RECREACAO                           5503
SIDERURGICA-

As variáveis ['setor',  'nm_divisao', 'nm_segmento'] possuem alguns valores faltantes! Vamos removê-las do DB pois estas empresas apresentam  muitas informações faltantes (elas também não estão presentes nos portfolios dos nossos usuários).

In [146]:
df.loc[df[ 'setor'].isna(), [ 'setor','de_ramo', 'nm_divisao', 'nm_segmento'] ].head(5)

,setor,de_ramo,nm_divisao,nm_segmento
156,NaN,OUTROS,NaN,NaN
165,NaN,OUTROS,NaN,NaN
372,NaN,OUTROS,NaN,NaN
554,NaN,OUTROS,NaN,NaN
620,NaN,OUTROS,NaN,NaN


Somente os ```setor``` de 'SERVIÇO' e 'INDUSTRIA' apresentam diferentes ```nm_segmento``` 

In [198]:
contigency_table(df,[['nm_divisao'],['setor', 'nm_segmento']],True)

## Avaliando anos

'dt_situacao' é a data de registro da empresa, porém temos tanto a idade em anos quanto em categorias. Vamos seguir somente a variável categórica e remover as demais.

In [148]:
to_drop_age = ['idade_empresa_anos', 'dt_situacao']

df[['idade_empresa_anos', 'idade_emp_cat','dt_situacao']].tail(5)

,idade_empresa_anos,idade_emp_cat,dt_situacao
462293,3.254795,1 a 5,2015-07-28
462294,8.953425,5 a 10,2009-11-16
462295,20.745205,> 20,2005-11-03
462296,1.909589,1 a 5,2016-11-30
462297,0.797260,<= 1,2018-01-10


## Avaliando saúde das empresas

'nu_meses_rescencia' parece estar representado pela variável 'de_saude_rescencia'. Vamos removê-la!

In [149]:
for i in ['de_saude_tributaria', 'de_saude_rescencia', 'de_nivel_atividade']:
    print(i, df[i].nunique())
    print(df[i].value_counts(dropna=False))
    print()

de_saude_tributaria 6
VERDE       145430
AZUL        105230
AMARELO      91045
CINZA        64019
LARANJA      36116
NaN          14851
VERMELHO      5607
Name: de_saude_tributaria, dtype: int64

de_saude_rescencia 5
ACIMA DE 1 ANO    378896
ATE 1 ANO          38101
SEM INFORMACAO     30425
NaN                14851
ATE 3 MESES           13
ATE 6 MESES           12
Name: de_saude_rescencia, dtype: int64

de_nivel_atividade 4
MEDIA          217949
ALTA           152245
BAIXA           76080
NaN             11168
MUITO BAIXA      4856
Name: de_nivel_atividade, dtype: int64



In [150]:
to_drop_rescencia = ['nu_meses_rescencia']

df[['de_saude_tributaria', 'de_saude_rescencia', 'nu_meses_rescencia', 'de_nivel_atividade', 'fl_simples_irregular', 'fl_st_especial']].tail(5)

,de_saude_tributaria,de_saude_rescencia,nu_meses_rescencia,de_nivel_atividade,fl_simples_irregular,fl_st_especial
462293,VERDE,ACIMA DE 1 ANO,22.0,ALTA,False,False
462294,LARANJA,ACIMA DE 1 ANO,23.0,MUITO BAIXA,False,False
462295,AMARELO,ACIMA DE 1 ANO,23.0,MEDIA,False,False
462296,VERDE,ACIMA DE 1 ANO,20.0,ALTA,False,False
462297,CINZA,SEM INFORMACAO,NaN,BAIXA,False,False


In [199]:
contigency_table(df,[['de_saude_tributaria'], ['de_saude_rescencia','de_nivel_atividade']], ax = 1)

### Estas variáveis são muito relevantes para aprimorar a seleção de leads!

de_saude_tributaria: character, indicator of health tax status, Green if all tax are OK, Red if none are OK

de_saude_rescencia:character, indicates time of update of the most lagged input of the indicator

de_nivel_atividade:character, probability of being operating, ALTA high probality, BAIXA, low probality

fl_st_especial: boolean value, true if dt_situacao_especial is not null. If it is not null means that some
extraordinary situation is identified by the IRS (ESPOLIO DE EMPRESARIO EMPRESA INDIVIDUAL OU
EIRELI, FALIDO, EM LIQUIDACAO, LIQUIDACAO JUDICIAL, LIQUIDACAO EXTRA JUDICIAL, REGISTRO NA
JUNTA COMERCIAL EM ANDAMENTO, EM LIQUID EXTRA JUDICIAL, RECUPERACAO JUDICIAL,
INTERVENCAO)

## Avaliando características regionais

In [152]:
df[['empsetorcensitariofaixarendapopulacao', 'nm_meso_regiao', 'nm_micro_regiao']].tail(5)

,empsetorcensitariofaixarendapopulacao,nm_meso_regiao,nm_micro_regiao
462293,585.63,AGRESTE POTIGUAR,BORBOREMA POTIGUAR
462294,2429.89,CENTRO NORTE PIAUIENSE,TERESINA
462295,3150.81,NORTE MARANHENSE,AGLOMERACAO URBANA DE SAO LUIS
462296,2752.49,NaN,NaN
462297,NaN,NaN,NaN


In [153]:
df['empsetorcensitariofaixarendapopulacao']

0             NaN
1          468.93
2          713.50
3          475.73
4         2874.04
           ...   
462293     585.63
462294    2429.89
462295    3150.81
462296    2752.49
462297        NaN
Name: empsetorcensitariofaixarendapopulacao, Length: 462298, dtype: float64

In [208]:
values_contigency_table(df, [['nm_meso_regiao', 'nm_micro_regiao'],['sg_uf']], values = 'empsetorcensitariofaixarendapopulacao', aggfunc = 'mean', na=True,ax=0)

## Avaliando informações sobre os Sócios

Entede-se que estas informações são relevantes mas podem ser consideradas excessivas (repetitivas) e podem pesar na escolha dos leads. Portanto, para reduzir dimensionalidade decidimos excluir algumas.

In [95]:
to_drop_idade = ['qt_socios', 'idade_maxima_socios', 'idade_minima_socios']

df.filter(like='socios').tail(5)

,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,idade_maxima_socios,idade_minima_socios,qt_socios_st_regular
462293,1.0,1.0,0.0,31.000000,31.0,31.0,1.0
462294,NaN,NaN,NaN,NaN,NaN,NaN,NaN
462295,1.0,1.0,0.0,74.000000,74.0,74.0,1.0
462296,5.0,5.0,0.0,31.666667,39.0,25.0,3.0
462297,1.0,1.0,0.0,44.000000,44.0,44.0,1.0


In [227]:
contigency_table(df[df['qt_socios']<20], [['de_natureza_juridica'],['qt_socios']],ax = 1)

qt_socios,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,Total
de_natureza_juridica,,,,,,,,,,,,,,,,,,,,
ASSOCIACAO PRIVADA,28117,829,291,134,56,70,22,13,14,14,8,16,7,1,1,1,1,0,2,29597
CONSORCIO DE SOCIEDADES,0,2,110,31,6,0,0,2,0,0,1,0,1,0,0,0,0,0,0,153
COOPERATIVA,255,384,222,82,66,54,19,7,11,8,5,7,3,5,2,1,1,3,1,1136
EMPRESA INDIVIDUAL DE RESPONSABILIDADE LIMITADA DE NATUREZA EMPRESARIA,14752,230,8,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14991
EMPRESA INDIVIDUAL DE RESPONSABILIDADE LIMITADA DE NATUREZA SIMPLES,79,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,82
EMPRESA PUBLICA,10,5,0,8,40,4,0,479,0,0,0,0,0,0,0,0,0,0,0,546
EMPRESARIO INDIVIDUAL,207517,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,207518
ESTABELECIMENTO NO BRASIL DE SOCIEDADE ESTRANGEIRA,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5
FUNDACAO PRIVADA,280,75,67,23,6,4,5,1,0,1,1,0,0,0,3,1,0,1,0,468


## Avaliando informações sobre faturamento

As variáveis '_aux' parecem ser uma subclasse especificando melhor o valor faturado dentro de cada faixa. No momento optamos por reduzir informaçoes "duplicadas".

In [96]:
to_drop_faturamento = ['vl_faturamento_estimado_aux','vl_faturamento_estimado_grupo_aux']

df.filter(like='faturamento').tail(5)

,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,vl_faturamento_estimado_aux,vl_faturamento_estimado_grupo_aux
462293,"ATE R$ 81.000,00","ATE R$ 81.000,00",50000.0,50000.0
462294,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 81.000,01 A R$ 360.000,00",210000.0,210000.0
462295,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 81.000,01 A R$ 360.000,00",210000.0,210000.0
462296,"DE R$ 360.000,01 A R$ 1.500.000,00","DE R$ 360.000,01 A R$ 1.500.000,00",432734.4,432734.4
462297,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 81.000,01 A R$ 360.000,00",210000.0,210000.0


In [231]:
contigency_table(df, [['natureza_juridica_macro'],['de_faixa_faturamento_estimado']], False, ax = 1)

de_faixa_faturamento_estimado,ACIMA DE 1 BILHAO DE REAIS,"ATE R$ 81.000,00","DE R$ 1.500.000,01 A R$ 4.800.000,00","DE R$ 10.000.000,01 A R$ 30.000.000,00","DE R$ 100.000.000,01 A R$ 300.000.000,00","DE R$ 30.000.000,01 A R$ 100.000.000,00","DE R$ 300.000.000,01 A R$ 500.000.000,00","DE R$ 360.000,01 A R$ 1.500.000,00","DE R$ 4.800.000,01 A R$ 10.000.000,00","DE R$ 500.000.000,01 A 1 BILHAO DE REAIS","DE R$ 81.000,01 A R$ 360.000,00",SEM INFORMACAO,Total
natureza_juridica_macro,,,,,,,,,,,,,
ADMINISTRACAO PUBLICA,12,9,107,79,41,66,18,270,48,12,2053,3,2912
CARGO POLITICO,0,0,0,0,0,0,0,0,0,0,12,0,1617
ENTIDADES EMPRESARIAIS,5,1284,6447,1061,130,558,15,18541,1602,11,67204,685,99241
ENTIDADES SEM FINS LUCRATIVOS,0,276,433,84,9,44,4,8176,93,0,27363,44,37089
INSTITUICOES EXTRATERRITORIAIS,0,0,0,0,0,0,0,0,0,0,3,0,3
OUTROS,4,96984,3164,221,73,203,11,19215,407,4,176031,441,320211
PESSOAS FISICAS,0,5,2,0,0,0,0,23,0,0,1195,0,1225
Total,21,98558,10153,1445,253,871,48,46225,2150,27,273861,1173,462298


## Clean data

In [294]:
max_percentual_na = 40
df = dropna_cols(df_raw,info_DB,max_percentual_na).copy()

In [363]:
## exclui empresas sem 'setor',  'nm_divisao', 'nm_segmento'

print('Numero de empresas sem informações básicas (setor, nm_divisao, nm_segmento)',
df.loc[df.setor.isna(),['setor','de_ramo', 'nm_divisao', 'nm_segmento']].shape[0])

df.dropna(subset = ['setor',  'nm_divisao', 'nm_segmento'], axis = 0, inplace = True)

Numero de empresas sem informações básicas (setor, nm_divisao, nm_segmento) 1927


In [364]:
to_drop_age = ['idade_empresa_anos', 'dt_situacao']
to_drop_rescencia = ['nu_meses_rescencia']
to_drop_idade = ['qt_socios', 'idade_maxima_socios', 'idade_minima_socios']
to_drop_natureza = ['fl_optante_simei', 'fl_optante_simples']#['fl_me','fl_sa', 'fl_epp', 'fl_mei', 'fl_ltda']
to_drop_faturamento = ['vl_faturamento_estimado_aux','vl_faturamento_estimado_grupo_aux']
to_drop_extra = [ 'fl_email','fl_telefone','sg_uf_matriz']

to_drop = to_drop_age + to_drop_rescencia + to_drop_idade + to_drop_natureza + to_drop_faturamento + to_drop_extra

print('N cols to drop', len(to_drop))

df.drop(columns = to_drop, inplace=True)

N cols to drop 13


## Imputation of missing values

In [365]:
to_fill = ['de_faixa_faturamento_estimado','de_faixa_faturamento_estimado_grupo',
'de_saude_tributaria','de_nivel_atividade', 'de_saude_rescencia','nm_meso_regiao','nm_micro_regiao']

for x in to_fill:
    df[x].fillna('SEM INFORMACAO', inplace=True)# Fill nominal/ordinal categorical data with 'SEM INFORMACAO'
to_fill = ['de_faixa_faturamento_estimado','de_faixa_faturamento_estimado_grupo',
'de_nivel_atividade', 'de_saude_rescencia','nm_meso_regiao','nm_micro_regiao']

df['de_saude_tributaria'].fillna('CINZA')

for x in to_fill:
    df[x].fillna('SEM INFORMACAO', inplace=True)

In [366]:
# Fill numeric values with the mean\median
df['idade_media_socios'].fillna(int(df['idade_media_socios'].mean()), inplace=True)
df['empsetorcensitariofaixarendapopulacao'].fillna(df['empsetorcensitariofaixarendapopulacao'].median(), inplace=True)
df['empsetorcensitariofaixarendapopulacao'] = np.log(df['empsetorcensitariofaixarendapopulacao'])

In [367]:
# Fill these with bussiness info
df['qt_socios_pf'].fillna(1, inplace=True)
df['qt_socios_pj'].fillna(0, inplace=True)
df['qt_socios_st_regular'].fillna(df['qt_socios_pf'], inplace=True)

Vamos verificar a situação do nosso dataframe.

In [368]:
df_info = get_info(df)
df_info[df_info['nan']!=0]

Shape ::  (460371, 37)


,types,nan,nan%,unique


In [369]:
features, categ_features, num_features, categ_bool = categorical_types(df)  

(460371, 37) 

Num:['vl_total_veiculos_pesados_grupo', 'vl_total_veiculos_leves_grupo', 'empsetorcensitariofaixarendapopulacao', 'qt_socios_pf', 'qt_socios_pj', 'idade_media_socios', 'qt_socios_st_regular', 'qt_filiais'] 

Categ: ['id', 'fl_matriz', 'de_natureza_juridica', 'sg_uf', 'natureza_juridica_macro', 'de_ramo', 'setor', 'idade_emp_cat', 'fl_me', 'fl_sa', 'fl_epp', 'fl_mei', 'fl_ltda', 'fl_st_especial', 'fl_rm', 'nm_divisao', 'nm_segmento', 'fl_spa', 'fl_antt', 'fl_veiculo', 'de_saude_tributaria', 'de_saude_rescencia', 'de_nivel_atividade', 'fl_simples_irregular', 'nm_meso_regiao', 'nm_micro_regiao', 'fl_passivel_iss', 'de_faixa_faturamento_estimado', 'de_faixa_faturamento_estimado_grupo']


### Features Numéricas

In [370]:
numerical_features = num_features
numerical_features

['vl_total_veiculos_pesados_grupo',
 'vl_total_veiculos_leves_grupo',
 'empsetorcensitariofaixarendapopulacao',
 'qt_socios_pf',
 'qt_socios_pj',
 'idade_media_socios',
 'qt_socios_st_regular',
 'qt_filiais']

### Features boolean

In [371]:
dic_bool = {
    'SIM': 1,
    'NAO': 0}

### Features ordinais

In [372]:
ordinal_features = [
    'de_faixa_faturamento_estimado',
    'de_faixa_faturamento_estimado_grupo',
    'idade_emp_cat',
    'de_saude_tributaria',
    'de_saude_rescencia',
    'de_nivel_atividade']

In [373]:
dic_faturamento_estimado = {
    'SEM INFORMACAO' : 0,
    'ATE R$ 81.000,00' : 1,
    'DE R$ 81.000,01 A R$ 360.000,00' : 2,
    'DE R$ 360.000,01 A R$ 1.500.000,00':3,
    'DE R$ 1.500.000,01 A R$ 4.800.000,00':4,
    'DE R$ 4.800.000,01 A R$ 10.000.000,00':5,
    'DE R$ 10.000.000,01 A R$ 30.000.000,00':6, 
    'DE R$ 30.000.000,01 A R$ 100.000.000,00':7,
    'DE R$ 100.000.000,01 A R$ 300.000.000,00':8,
    'DE R$ 300.000.000,01 A R$ 500.000.000,00':9,
    'DE R$ 500.000.000,01 A 1 BILHAO DE REAIS':10,
    'ACIMA DE 1 BILHAO DE REAIS':11}

In [374]:
dic_idade = {
    '<= 1': 1,
    '1 a 5': 2,
    '5 a 10': 3,
    '10 a 15':4,
    '15 a 20':5,
    '> 20' :6}

In [375]:
dic_de_nivel_atividade = {
    'SEM INFORMACAO': 0,
    'MUITO BAIXA': 1,
    'BAIXA':2,
    'MEDIA':3,
    'ALTA': 4
    }

In [376]:
dic_de_saude_rescencia = {'SEM INFORMACAO': 0,
'ATE 3 MESES' : 1,
'ATE 6 MESES' :2,
'ATE 1 ANO':3,
'ACIMA DE 1 ANO':4
}

In [377]:
dic_de_saude_tributaria = {
  'CINZA': 0,
  'VERMELHO':1,
  'LARANJA': 2,
  'AMARELO':3,
  'AZUL':4,
  'VERDE':5}

### Features categoricas

In [378]:
categorical_features = [
    'natureza_juridica_macro',
    'de_natureza_juridica',
    'de_ramo',
    'setor',
    'nm_divisao',
    'nm_segmento',
    'nm_meso_regiao',
    'nm_micro_regiao',
    'sg_uf']

In [379]:
len(categorical_features + ordinal_features + bool_features + numerical_features)

36

## Encode boolean

In [380]:
# Encode the boolean features
df['fl_rm'] = df['fl_rm'].map(dic_bool)

bool_features = [col for col in df.columns if col.startswith('fl_')]

# Treat them as integers
for feat in bool_features:    
    df[feat] = df[feat].astype(int)

## Encode Ordinals

In [381]:
df['de_faixa_faturamento_estimado'] = df['de_faixa_faturamento_estimado'].map(dic_faturamento_estimado)
df['de_faixa_faturamento_estimado_grupo'] = df['de_faixa_faturamento_estimado_grupo'].map(dic_faturamento_estimado)
df['idade_emp_cat'] = df['idade_emp_cat'].map(dic_idade)
df['de_nivel_atividade'] = df['de_nivel_atividade'].map(dic_de_nivel_atividade)
df['de_saude_rescencia'] = df['de_saude_rescencia'].map(dic_de_saude_rescencia)
df['de_saude_tributaria'] = df['de_saude_tributaria'].map(dic_de_saude_tributaria)

## Encode categoricals

In [382]:
map_categories = dict()
for feat in categorical_features:
    df[feat] = df[feat].astype('category')
    encode_categ = df[feat].cat.codes
    map_categories[feat] = dict(zip(encode_categ, df[feat]))
    df[feat] = encode_categ

In [383]:
[x for x in categ_features if x not in bool_features + ordinal_features]

['id',
 'de_natureza_juridica',
 'sg_uf',
 'natureza_juridica_macro',
 'de_ramo',
 'setor',
 'nm_divisao',
 'nm_segmento',
 'nm_meso_regiao',
 'nm_micro_regiao']

In [384]:
df.iloc[3,:].T

id                                       3348900fe63216a439d2e5238c79ddd46ede454df7b9d8...
fl_matriz                                                                                1
de_natureza_juridica                                                                    17
sg_uf                                                                                    1
natureza_juridica_macro                                                                  5
de_ramo                                                                                 24
setor                                                                                    4
idade_emp_cat                                                                            3
fl_me                                                                                    0
fl_sa                                                                                    0
fl_epp                                                                                   0

In [385]:
get_info(df)

Shape ::  (460371, 37)


,types,nan,nan%,unique
id,object,0,0.0,460371
fl_matriz,int32,0,0.0,2
de_natureza_juridica,int8,0,0.0,66
sg_uf,int8,0,0.0,6
natureza_juridica_macro,int8,0,0.0,7
de_ramo,int8,0,0.0,32
setor,int8,0,0.0,5
idade_emp_cat,int64,0,0.0,6
fl_me,int32,0,0.0,2
fl_sa,int32,0,0.0,2


In [386]:
df.columns.to_list()

['id',
 'fl_matriz',
 'de_natureza_juridica',
 'sg_uf',
 'natureza_juridica_macro',
 'de_ramo',
 'setor',
 'idade_emp_cat',
 'fl_me',
 'fl_sa',
 'fl_epp',
 'fl_mei',
 'fl_ltda',
 'fl_st_especial',
 'fl_rm',
 'nm_divisao',
 'nm_segmento',
 'fl_spa',
 'fl_antt',
 'fl_veiculo',
 'vl_total_veiculos_pesados_grupo',
 'vl_total_veiculos_leves_grupo',
 'de_saude_tributaria',
 'de_saude_rescencia',
 'de_nivel_atividade',
 'fl_simples_irregular',
 'empsetorcensitariofaixarendapopulacao',
 'nm_meso_regiao',
 'nm_micro_regiao',
 'fl_passivel_iss',
 'qt_socios_pf',
 'qt_socios_pj',
 'idade_media_socios',
 'qt_socios_st_regular',
 'de_faixa_faturamento_estimado',
 'de_faixa_faturamento_estimado_grupo',
 'qt_filiais']

In [318]:
df.to_csv('estaticos_market_clean.csv')